In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
### Implement a function that take's a recipe name (e.g. 'Shrimp and Pineapple Stir-Fry')
### returns all ingredients in a list, returns each step in instructions

def get_recipe(recipe_name):
    
    ingredient_list = recipe_text_df[recipe_text_df['title'] == recipe_name]['recipe']\
                                                    .values[0]['Ingredients']
    print("Ingredients:")
    print("----------------------")
    for ingredient in ingredient_list:
        print(ingredient)
    print()
        
    steps = recipe_text_df[recipe_text_df['title'] == recipe_name]['recipe']\
                                            .values[0]['Instructions']
    
    for x in range(len(steps)):
        step = x + 1
        print(f"Step {step}:")
        print("----------------------")
        print(steps[x][f"Step {step}"])
        print("----------------------")
        

In [3]:
def recipe_scrape(url):
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    ingredient_tags = soup.find_all('li', class_ = "mntl-structured-ingredients__list-item")
    ingredient_list = []
    for ingredient in ingredient_tags:
        span_tags = ingredient.find_all('span')
        try:
            quantity = span_tags[0].text
        except:
            print('No quantity value')
            break
        try:
            unit = span_tags[1].text
        except:
            print('No unit value')
            break
        try:
            name = span_tags[2].text
        except:
            print('No name value')
            ingredient = f"{quantity} {unit}"
        
        ingredient = f"{quantity} {unit} {name}"
        
        ingredient_list.append(ingredient)
    cooking_instructions = soup.find_all('li', class_ = "comp mntl-sc-block-group--LI mntl-sc-block mntl-sc-block-startgroup")
    
    instr_list = []
    step = 1
    for instructions in cooking_instructions:
        step_label = f"Step {step}"
        instr_dict = {}
        cooking_instructions_text = instructions.find('p').text.strip()

        instr_dict[step_label] = cooking_instructions_text
        instr_list.append(instr_dict)
        step += 1
    
    dict = {}
    dict['Ingredients'] = ingredient_list
    dict['Instructions'] = instr_list
    
    return dict

In [4]:
def recipe_search(search_string,num_of_pages):
    from splinter import Browser
    from bs4 import BeautifulSoup
    import pandas as pd
    import time
    
    recipe_list = []
    title_list = []
    recipe_text_list = []
    string = search_string.replace(' ','+')
    url = f'https://www.allrecipes.com/search?q={string}'
    
    browser.visit(url)
    
    x = range(num_of_pages)
    for _ in x:
        print("-------------------")
        print(f"Starting scrape for page {_+1}")
        print("-------------------")
        
        html = browser.html


        article_soup = BeautifulSoup(html, 'html.parser')
        
        links = article_soup.find_all('a', class_='comp mntl-card-list-items mntl-document-card mntl-card card card--no-image')
        for link in links:
            ratings = link.find_all('div', class_ = 'comp recipe-card-meta')
            if len(ratings) > 0:
                recipe_dict = {}
                recipe_text_dict = {}
                print('found recipe')
                print('storing recipe...')
                
                title = link.find(class_ = 'card__title-text').text
                rating = len(link.find_all(class_ = 'icon icon-star'))
                recipe_url = link.get('href')
                
                recipe_dict['title'] = title
                recipe_dict['rating'] = rating
                recipe_dict['url'] = recipe_url
                                    
                if title in title_list:
                    print('Recipe already found in dictionary')
                else:
                    title_list.append(title)
                    
                    recipe_text_dict['title'] = title
                    recipe_text_dict['recipe'] = recipe_scrape(recipe_url)
                    recipe_text_list.append(recipe_text_dict)
                    
                recipe_list.append(recipe_dict)
            
            else:
                print('article found..skipping')
                
        browser.visit(url)       
        
        if _ == len(x)-1:
            print("-------------------")
            print('Reached the last page...')
            print('All recipes scraped and stored...')
            print("-------------------")
        else:
            try:
                print(browser.url)
                time.sleep(2)
                browser.links.find_by_partial_text('Next').click()
                time.sleep(2)
               
                print("-------------------")
                print("Moving to next page")
                print("-------------------")
                
                print(browser.url)
                url = browser.url
            except:
                print("-------------------")
                print('Reached Except Clause for Next Button...')
                print('All recipes scraped and stored...')
                print("-------------------")
                
                recipe_text_df = pd.DataFrame(recipe_text_list)
                recipe_df = pd.DataFrame(recipe_list)
                recipe_df = recipe_df.drop_duplicates()
                top_recipe_df = recipe_df[recipe_df['rating'] == 5]
                

                return top_recipe_df,recipe_text_df

    recipe_df = pd.DataFrame(recipe_list)
    try:
        recipe_df = recipe_df.drop_duplicates()
    except:
        print('no duplicates')
    top_recipe_df = recipe_df[recipe_df['rating'] == 5]
    
    
    recipe_text_df = pd.DataFrame(recipe_text_list)
    browser.quit()
    return top_recipe_df, recipe_text_df

In [6]:
browser = Browser('chrome')

In [7]:
#use recipe string and number of search pages as arguements
#return two pandas DataFrames (all recipes/five-star recipes)
top_recipes_df, recipe_text_df = recipe_search('tacos',4)

-------------------
Starting scrape for page 1
-------------------
article found..skipping
found recipe
storing recipe...
found recipe
storing recipe...
article found..skipping
found recipe
storing recipe...
article found..skipping
found recipe
storing recipe...
found recipe
storing recipe...
article found..skipping
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
article found..skipping
article found..skipping
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
found recipe
storing recipe...
https://www.allrecipes.com/search?q=tacos
-------------------
Moving to next page
-------------------
https://www.allrecipes.com/search?tacos=tacos&offset=24&q=tacos
-------------------
Starting scrape for page 2
------------------

In [8]:
#print top_recipes df here
top_recipes_df

,title,rating,url
3,Turkey and Yam Spicy Tacos,5,https://www.allrecipes.com/recipe/216244/turke...
19,Simple Slow-Cooked Korean Beef Soft Tacos,5,https://www.allrecipes.com/recipe/237005/simpl...
23,Birria de Res Tacos (Beef Birria Tacos),5,https://www.allrecipes.com/recipe/265338/birri...
48,Healthy Fish Tacos with Mango Salsa,5,https://www.allrecipes.com/recipe/233899/healt...
52,Shrimp Tacos with Cilantro-Lime Crema,5,https://www.allrecipes.com/recipe/280916/shrim...
56,Tacos de Papa (Potato Tacos),5,https://www.allrecipes.com/recipe/255418/tacos...


In [9]:
get_recipe("Healthy Fish Tacos with Mango Salsa")

Ingredients:
----------------------
1  mango - peeled, seeded, and chopped
2  avocados - peeled, pitted, and chopped
1 cup chopped tomato
½ cup diced red onion
½ cup diced red pepper
2 tablespoons chopped fresh flat-leaf parsley
2 tablespoons canola oil
2 tablespoons brown sugar
1  lime, juiced
1 teaspoon cider vinegar
1 dash hot pepper sauce, or to taste
  salt and ground black pepper to taste
8  taco shells
4 (6 ounce) mahi mahi fillets, cut into 1-inch slices
1 teaspoon ground black pepper
1 teaspoon paprika
½ teaspoon salt
2 tablespoons olive oil

Step 1:
----------------------
Stir mango, avocado, tomato, red onion, red pepper, parsley, canola oil, brown sugar, lime juice, cider vinegar, hot pepper sauce, salt, and ground black pepper together in a bowl. Refrigerate salsa for at least 1 hour.
----------------------
Step 2:
----------------------
Preheat oven to 325 degrees F (165 degrees C).
----------------------
Step 3:
----------------------
Heat taco shells in preheated oven u

In [10]:
browser.quit()